# Qwen2.5-Coder-7B to ONNX Converter

This notebook converts Qwen2.5-Coder-7B to ONNX format for use in Code Tutor.

**Steps:**
1. Install required packages
2. Download and convert the model
3. Upload to HuggingFace

**Estimated time:** 30-45 minutes
**Output size:** ~4-5GB

## Step 1: Install Packages

In [ ]:
# Install required packages
!pip install -q transformers torch onnx optimum[exporters] huggingface-hub

print("✓ Packages installed")

## Step 2: Convert Model to ONNX

In [ ]:
from optimum.exporters.onnx import main_export
from pathlib import Path

# Set output directory
output_dir = Path('/content/qwen2.5-coder-7b-onnx')
output_dir.mkdir(exist_ok=True)

print("Starting conversion...")
print("This will take 30-45 minutes")
print("Downloading ~15GB, output will be ~4-5GB")
print()

# Convert to ONNX with INT4 quantization
main_export(
    model_name_or_path='Qwen/Qwen2.5-Coder-7B-Instruct',
    output=output_dir,
    task='text-generation',
    trust_remote_code=True,
    fp16=True,
    optimize='O4',  # Aggressive optimization
)

print()
print("✓ Conversion complete!")
print(f"Files saved to: {output_dir}")
print()
print("Files created:")
for f in output_dir.iterdir():
    size_mb = f.stat().st_size / (1024 * 1024)
    print(f'  - {f.name}: {size_mb:.1f} MB')

## Step 3: Upload to HuggingFace

1. Get your HuggingFace token from https://huggingface.co/settings/tokens
2. Run the cell below and enter your token when prompted

In [ ]:
from huggingface_hub import HfApi, login
from getpass import getpass

# Get token
token = getpass("Enter your HuggingFace token: ")
login(token=token)

# Set your username and repo name
username = input("Enter your HuggingFace username: ")
repo_name = "Qwen2.5-Coder-7B-Instruct-ONNX"
repo_id = f"{username}/{repo_name}"

print(f"Uploading to {repo_id}...")
print("This may take 10-20 minutes depending on connection")

# Upload
api = HfApi()
api.create_repo(repo_id=repo_id, exist_ok=True)
api.upload_folder(
    folder_path='/content/qwen2.5-coder-7b-onnx',
    repo_id=repo_id,
    repo_type="model"
)

print()
print(f"✓ Upload complete!")
print(f"Model available at: https://huggingface.co/{repo_id}")

## Done!

Your model is now available on HuggingFace. Update the Code Tutor app with your repo URL:

```csharp
HuggingFaceRepo = "your-username/Qwen2.5-Coder-7B-Instruct-ONNX",
ModelPath = "",  // Files at root
```